In [2]:
pip install openai python-dotenv datasets

     -------------------------------------- 755.0/755.0 KB 3.7 MB/s eta 0:00:00
     -------------------------------------- 491.5/491.5 KB 4.4 MB/s eta 0:00:00
     -------------------------------------- 100.9/100.9 KB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 73.5/73.5 KB ? eta 0:00:00
     -------------------------------------- 208.8/208.8 KB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 KB 4.3 MB/s eta 0:00:00
     -------------------------------------- 444.8/444.8 KB 4.7 MB/s eta 0:00:00
     -------------------------------------- 515.4/515.4 KB 6.5 MB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 KB 6.6 MB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 KB 3.8 MB/s eta 0:00:00
     -------------------------------------- 162.3/162.3 KB 4.9 MB/s eta 0:00:00
     ------------------------------------- 193.6/193.6 KB 12.2 MB/s eta 0:00:00
     -------------------------------------- 451

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from datasets import load_dataset # datasets 라이브러리 임포트

# .env 파일 로드
load_dotenv()

client = None
try:
    client = OpenAI()
    print("OpenAI 클라이언트가 성공적으로 초기화되었습니다.")
    # (선택 사항) API 연결 확인을 위한 간단한 테스트:
    # models_response = client.models.list()
    # print(f"API 연결 확인. 사용 가능한 모델 예시: {[m.id for m in models_response.data[:3]]}")
except Exception as e:
    print(f"오류: OpenAI 클라이언트 초기화 실패: {e}") # 실제 오류 메시지 출력
    print(f"'.env' 파일에 'OPENAI_API_KEY'가 설정되었는지, 또는 키가 유효한지 확인해주세요.")
    print("이 오류가 발생하면 이후 감정 분석 기능이 정상 작동하지 않습니다.")

def analyze_sentiment(user_text: str) -> str:
    """
    사용자의 텍스트(한국어 또는 영어)를 입력받아 GPT API를 통해 감정을 분석하고
    미리 정의된 한국어 감정 목록 중 하나로 반환하는 함수.
    이 함수는 go_emotions 데이터셋의 few-shot 예시를 활용합니다.

    Args:
        user_text (str): 분석할 사용자의 대화 텍스트 (한국어 또는 영어).

    Returns:
        str: 분석된 감정 (예: '기쁨', '슬픔', '중립' 등).
             GPT가 정해진 감정 외 다른 것을 반환하거나, API 호출에 실패할 경우 '중립' 또는 '오류' 반환.
    """
    if client is None:
        print("경고: OpenAI 클라이언트가 초기화되지 않아 감정 분석을 수행할 수 없습니다. API 키 설정을 확인해주세요.")
        return "오류"

    allowed_emotions = [
        # 🌞 긍정
        "기쁨","환희","유쾌","희열","만족","뿌듯함","감사","사랑","애정",
        "평온","안도","자신감","설렘","기대","존경","경외","흥분","열정","유머", # 열정, 유머 추가됨
        # 🌧 부정
        "슬픔","우울","상실감","실망","좌절","후회","자책","외로움","분노","짜증",
        "혐오","경멸","두려움","공포","불안","초조","수치","부끄러움","피로","무기력","억울함", # 무기력, 억울함 추가됨
        # 🌪 혼합·복합
        "혼란","갈등","당황","놀람","충격","호기심","의심","불신","그리움","향수", # 충격, 불신, 향수 추가됨
        # 💤 기타
        "중립"
    ]

    # --- GoEmotions 데이터셋 로드 및 전처리 ---
    # 데이터셋의 English 감정 레이블을 한국어 레이블로 매핑
    # 이 매핑은 GoEmotions 데이터셋의 28개 감정 레이블을 기반으로 합니다.
    # 각 영문 감정에 가장 적합한 한국어 감정을 매칭시켜야 합니다.
    # allowed_emotions 리스트에 없는 감정은 '중립' 등으로 폴백 처리하거나,
    # allowed_emotions 리스트를 확장해야 합니다.
    go_emotions_to_korean_map = {
        'admiration': '존경',
        'amusement': '유머',
        'anger': '분노',
        'annoyance': '짜증',
        'approval': '만족',
        'caring': '사랑',
        'confusion': '혼란',
        'curiosity': '호기심',
        'desire': '기대',
        'disappointment': '실망',
        'disapproval': '불신',
        'disgust': '혐오',
        'embarrassment': '부끄러움',
        'excitement': '흥분',
        'fear': '두려움',
        'gratitude': '감사',
        'grief': '슬픔', # 강한 슬픔
        'joy': '기쁨',
        'love': '사랑',
        'nervousness': '불안',
        'optimism': '자신감', # 또는 '기대'
        'pride': '뿌듯함',
        'realization': '놀람', # 또는 '깨달음', 문맥에 따라
        'relief': '안도',
        'remorse': '후회',
        'sadness': '슬픔',
        'surprise': '놀람',
        'neutral': '중립',
        'disappointment': '실망', # 중복 방지. 다른 감정과의 관계 확인 필요.
        'loneliness': '외로움',
        'shame': '수치',
        'fatigue': '피로',
        'emptiness': '무기력',
        'bewilderment': '당황',
        'agitation': '초조',
        'distress': '좌절',
        'longing': '그리움',
        'hope': '기대', # '설렘'과 비슷
        'triumph': '희열',
        'lively': '유쾌',
        'contentment': '평온'
    }

    try:
        # 데이터셋 로드
        ds = load_dataset("google-research-datasets/go_emotions", "simplified")
        # GoEmotions 데이터셋의 레이블 이름 목록 (영어)
        go_emotions_label_names = ds['train'].features['labels'].feature.names

        # 프롬프트 구성 시작
        prompt_messages = [
            {"role": "system", "content": f"너는 사용자의 감정을 분석하는 전문 심리 분석가야. 요청하는 문장에서 가장 지배적인 감정을 정확하게 식별하고, 오직 다음 한국어 감정 단어 목록 중 하나로만 반환해야 해: {', '.join(allowed_emotions)}. 만약 사용자 문장이 영어로 되어있다면, 분석된 영어 감정에 가장 가까운 한국어 감정 단어를 목록에서 찾아 반환해야 해. 다른 부가적인 설명, 문장, 인사는 일절 허용되지 않아."},
            {"role": "user", "content": "나는 너에게 한국어 또는 영어 문장을 제공할 거야. 너는 그 문장에서 가장 강하게 드러나는 감정을 위에 명시된 한국어 감정 단어 중 하나로, 오직 그 단어 자체만 출력해줘. 다른 불필요한 설명, 문장, 인사는 일절 포함하지 마."},
        ]

        # Few-shot 예시 데이터셋에서 가져오기
        # 너무 많은 예시를 넣으면 토큰 비용이 늘어나고, GPT가 혼란스러워할 수 있으므로 적절한 개수만 사용
        num_few_shot_examples = 100 # 사용할 few-shot 예시의 개수
        added_examples_count = 0
        for example in ds['train']:
            # go_emotions 'simplified'는 보통 단일 레이블을 가집니다.
            # 만약 다중 레이블이 있다면, 첫 번째 레이블만 사용하거나,
            # (예: if len(example['labels']) == 1: ) 단일 레이블 예시만 필터링할 수 있습니다.
            if example['labels'] and added_examples_count < num_few_shot_examples:
                original_english_label_id = example['labels'][0]
                original_english_label_name = go_emotions_label_names[original_english_label_id]
                
                # 영어 레이블을 한국어 레이블로 매핑
                korean_emotion = go_emotions_to_korean_map.get(original_english_label_name, "중립")
                
                # 매핑된 한국어 감정 단어가 allowed_emotions 목록에 있는지 확인
                if korean_emotion not in allowed_emotions:
                    # 만약 매핑된 감정이 허용된 목록에 없다면 "중립"으로 폴백 (오류 방지)
                    print(f"경고: 매핑된 감정 '{korean_emotion}'(원문: {original_english_label_name})이 허용된 목록에 없습니다. '중립'으로 처리합니다.")
                    korean_emotion = "중립"

                prompt_messages.append({"role": "user", "content": f"문장: '{example['text']}'"})
                prompt_messages.append({"role": "assistant", "content": korean_emotion})
                added_examples_count += 1
            elif added_examples_count >= num_few_shot_examples:
                break

    except Exception as e:
        print(f"데이터셋 로드 또는 Few-shot 예시 생성 중 오류 발생: {e}")
        # 데이터셋 로드 실패 시에도 최소한의 프롬프트 유지 (하드코딩 예시 사용)
        prompt_messages = [
            {"role": "system", "content": f"너는 사용자의 감정을 분석하는 전문 심리 분석가야. 요청하는 문장에서 가장 지배적인 감정을 정확하게 식별하고, 오직 다음 한국어 감정 단어 목록 중 하나로만 반환해야 해: {', '.join(allowed_emotions)}. 만약 사용자 문장이 영어로 되어있다면, 분석된 영어 감정에 가장 가까운 한국어 감정 단어를 목록에서 찾아 반환해야 해. 다른 부가적인 설명, 문장, 인사는 일절 허용되지 않아."},
            {"role": "user", "content": "나는 너에게 한국어 또는 영어 문장을 제공할 거야. 너는 그 문장에서 가장 강하게 드러나는 감정을 위에 명시된 한국어 감정 단어 중 하나로, 오직 그 단어 자체만 출력해줘. 다른 불필요한 설명, 문장, 인사는 일절 포함하지 마."},
            # 데이터셋 로드 실패 시 fallback으로 사용할 하드코딩 예시 (원래 코드의 예시들을 여기에 추가)
            {"role": "user", "content": "문장: '오늘 정말 기분이 최고야!'"}, {"role": "assistant", "content": "기쁨"},
            {"role": "user", "content": "문장: 'I feel incredibly happy today!'"}, {"role": "assistant", "content": "기쁨"},
            {"role": "user", "content": "문장: 'Why am I so sad?'"}, {"role": "assistant", "content": "슬픔"},
            {"role": "user", "content": "문장: 'This is completely neutral'"}, {"role": "assistant", "content": "중립"},
        ]
        print("데이터셋 로드에 실패하여 기본 하드코딩 예시로 전환합니다.")


    # 실제 분석 대상 문장을 prompt_messages 리스트에 추가
    prompt_messages.append({"role": "user", "content": f"문장: '{user_text}'"})

    # 사용할 GPT 모델 선택:
    # "gpt-4o" 또는 "gpt-4"가 언어 혼용 처리 및 정확한 감정 분석에 더 유리합니다.
    model_to_use = "gpt-4o"

    temperature = 0.0 # 0.0으로 하여 GPT가 지시를 가장 엄격하게 따르도록 함.
    max_tokens = 5 # 최소 토큰으로만 단어 하나만 생성하도록 제한.

    try:
        response = client.chat.completions.create(
            model=model_to_use,
            messages=prompt_messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        sentiment = response.choices[0].message.content.strip()

        if sentiment in allowed_emotions:
            return sentiment
        else:
            print(f"경고: GPT가 예상치 못한 감정 '{sentiment}'을(를) 반환했습니다. '중립'으로 처리합니다.")
            return "중립"

    except Exception as e:
        print(f"감정 분석 중 GPT API 호출 오류 발생: {e}")
        return "중립"

print("--- MBTI 상담 챗봇 단일 감정 분석 시작 ---")

while True:
    user_input = input("\n당신의 대화를 입력해주세요 (종료하려면 '종료' 입력): ")

    if user_input.lower() == '종료':
        print("감정 분석 프로그램을 종료합니다.")
        break

    print(f"\n[챗봇 분석 중...]")

    detected_emotion = analyze_sentiment(user_input)
    print(f"    > 분석된 감정: {detected_emotion}")
    print("-" * 50)

OpenAI 클라이언트가 성공적으로 초기화되었습니다.
--- MBTI 상담 챗봇 단일 감정 분석 시작 ---



[챗봇 분석 중...]
    > 분석된 감정: 중립
--------------------------------------------------
감정 분석 프로그램을 종료합니다.
